### 1. Configure Spanner CLI and create Shopify Database

In [1]:
%env GOPATH=/home/jupyter/go
%env PATH=$PATH:/usr/local/go/bin:$GOPATH/bin
%env SPANNER_EMULATOR_HOST=localhost:9010
%env CLI=/home/jupyter/go/bin/spanner-cli
%env SPAN_PROJECT={your project}
%env SPAN_INSTANCE=span-instance
%env SPAN_DATABASE=test-database

env: GOPATH=/home/jupyter/go
env: PATH=$PATH:/usr/local/go/bin:$GOPATH/bin
env: SPANNER_EMULATOR_HOST=localhost:9010
env: CLI=/home/jupyter/go/bin/spanner-cli
env: SPAN_PROJECT=cs327e-sp2020
env: SPAN_INSTANCE=span-instance
env: SPAN_DATABASE=test-database


In [2]:
!$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE -e "show tables"

Tables_in_test-database
T1


In [3]:
CLI_CONNECT="$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE"

In [14]:
!{CLI_CONNECT} -e "create database shopify"

In [15]:
!{CLI_CONNECT} -e "show databases"

Database
shopify
test-database


In [16]:
%env SPAN_DATABASE=shopify

env: SPAN_DATABASE=shopify


In [17]:
!{CLI_CONNECT} -f create_spanner_tables.sql

In [18]:
!{CLI_CONNECT} -e "show tables"

Tables_in_shopify
key_benefits
apps
pricing_plans
apps_categories
pricing_plan_features
categories
reviews


In [29]:
!{CLI_CONNECT} -e "show create table key_benefits"

Table	Create Table
key_benefits	CREATE TABLE key_benefits (
  app_id STRING(50),
  title STRING(2000),
  description STRING(MAX),
) PRIMARY KEY(app_id)


In [30]:
!{CLI_CONNECT} -e "show create table apps"

Table	Create Table
apps	CREATE TABLE apps (
  id STRING(50),
  url STRING(1000),
  title STRING(2000),
  developer STRING(100),
  developer_link STRING(1000),
  icon STRING(1000),
  rating FLOAT64,
  reviews_count INT64,
) PRIMARY KEY(id)


In [31]:
!{CLI_CONNECT} -e "show create table pricing_plans"

Table	Create Table
pricing_plans	CREATE TABLE pricing_plans (
  id STRING(50),
  app_id STRING(50),
  title STRING(200),
  price FLOAT64,
) PRIMARY KEY(id)


In [32]:
!{CLI_CONNECT} -e "show create table apps_categories"

Table	Create Table
apps_categories	CREATE TABLE apps_categories (
  app_id STRING(50),
  category_id STRING(50),
) PRIMARY KEY(app_id, category_id)


In [33]:
!{CLI_CONNECT} -e "show create table pricing_plan_features"

Table	Create Table
pricing_plan_features	CREATE TABLE pricing_plan_features (
  app_id STRING(50),
  pricing_plan_id STRING(50),
  features STRING(MAX),
) PRIMARY KEY(app_id, pricing_plan_id)


In [34]:
!{CLI_CONNECT} -e "show create table categories"

Table	Create Table
categories	CREATE TABLE categories (
  id STRING(50),
  title STRING(2000),
) PRIMARY KEY(id)


In [35]:
!{CLI_CONNECT} -e "show create table reviews"

Table	Create Table
reviews	CREATE TABLE reviews (
  app_id STRING(50),
  author STRING(100),
  rating INT64,
  date DATE,
) PRIMARY KEY(app_id, author)


#### 2. Open terminal and download shopify dataset: 
#####    gsutil cp gs://cs327e-open-access/shopify.zip .
#####    unzip shopify.zip

### 3. Populate tables

In [86]:
from google.cloud import spanner
from google.cloud.spanner_v1 import param_types
import pandas as pd

In [87]:
span_instance="span-instance"
span_database="shopify"
spanner_client = spanner.Client()
instance = spanner_client.instance(span_instance)
database = instance.database(span_database)

In [43]:
df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
values = df.values.tolist()

with database.batch() as batch:
    batch.insert(
        table="apps",
        columns=("id", "url", "title", "developer", "developer_link", "icon", "rating", "reviews_count"),
        values=values
    )

In [40]:
df = pd.read_csv('shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
values = df.values.tolist()

with database.batch() as batch:
    batch.insert(
        table="apps_categories",
        columns=("app_id", "category_id"),
        values=values
    )

In [45]:
df = pd.read_csv('shopify/categories.csv', sep=',', header=0, lineterminator='\n')
values = df.values.tolist()

with database.batch() as batch:
    batch.insert(
        table="categories",
        columns=("id", "title"),
        values=values
    )

In [49]:
df = pd.read_csv('shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
values = df.values.tolist()

with database.batch() as batch:
    batch.insert(
        table="key_benefits",
        columns=("app_id", "title", "description"),
        values=values
    )

In [52]:
df = pd.read_csv('shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
values = df.values.tolist()

with database.batch() as batch:
    batch.insert(
        table="pricing_plans",
        columns=("id", "app_id", "title", "price"),
        values=values
    )

In [55]:
df = pd.read_csv('shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
values = df.values.tolist()

with database.batch() as batch:
    batch.insert(
        table="pricing_plan_features",
        columns=("app_id", "pricing_plan_id", "feature"),
        values=values
    )

In [44]:
!{CLI_CONNECT} -e "select count(*) from apps"


3547


In [42]:
!{CLI_CONNECT} -e "select count(*) from apps_categories"


5383


In [46]:
!{CLI_CONNECT} -e "select count(*) from categories"


12


In [50]:
!{CLI_CONNECT} -e "select count(*) from key_benefits"


9541


In [56]:
!{CLI_CONNECT} -e "select count(*) from pricing_plans"


6275


In [57]:
!{CLI_CONNECT} -e "select count(*) from pricing_plan_features"


16270


In [95]:
def insert_app(transaction):

    row_ct = transaction.execute_update(
        "INSERT INTO apps (id, url, title, developer, developer_link, icon, reviews_count) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', "
          "'https://apps.shopify.com/watchlist?surface_detail=inventory-management&surface_inter_position=1&surface_intra_position=9&surface_type=category'," 
          "'Back in Stock Product Alerts', "
          "'Swym Corporation', "
          "'https://apps.shopify.com/partners/developer-ca6a967f09890f68',"
          "'https://apps.shopifycdn.com/listing_images/9905a4c8f22cb4a3b0c32af55a58ec21/icon/e6d46a7e5e1df375d542d033aae80459.png?height=72&width=72',"
           "0)"
    )
    
    print("{} record(s) inserted.".format(row_ct))
    
    row_ct = transaction.execute_update(
        "INSERT INTO apps_categories (app_id, category_id) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', '737ad50051083aa051d127a53b3ac0da')"
    )
    
    print("{} record(s) inserted.".format(row_ct))
    
    
    row_ct = transaction.execute_update(
        "INSERT INTO key_benefits (app_id, title, description) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', "
          "'Drive Back In Stock Sales', 'Back in stock, out of stock, pre order & restock alerts')"
    )
    
    print("{} record(s) inserted.".format(row_ct))
    
    row_ct = transaction.execute_update(
        "INSERT INTO pricing_plans (id, app_id, title, price) "
        "VALUES ('961a2751-1dbd-4f0f-8248-9fe6a6efd4ab', '683d06af-14c7-4733-9bde-ec5b699af996', "
                "'Free Trial', 14.99)"
    )
    
    print("{} record(s) inserted.".format(row_ct))
    

    row_ct = transaction.execute_update(
        "INSERT INTO pricing_plan_features (app_id, pricing_plan_id, feature) "
        " VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', '961a2751-1dbd-4f0f-8248-9fe6a6efd4ab', "
                 "'Up to 250 alert requests/mo')"
    )
    
    print("{} record(s) inserted.".format(row_ct))
      
database.run_in_transaction(insert_app)

1 record(s) inserted.
1 record(s) inserted.
1 record(s) inserted.
1 record(s) inserted.
1 record(s) inserted.


In [96]:
def update_price(transaction):
    row_ct = transaction.execute_update(
            "UPDATE pricing_plans "
            "SET price = price / 2 "
            "WHERE price > 0"
        )

    print("{} record(s) updated.".format(row_ct))

database.run_in_transaction(update_price)

4780 record(s) updated.


In [97]:
def delete_app(transaction):
        row_ct = transaction.execute_update(
            "DELETE FROM apps WHERE id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )

        print("{} record(s) deleted.".format(row_ct))
        
        row_ct = transaction.execute_update(
            "DELETE FROM apps_categories WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} record(s) deleted.".format(row_ct))
        
        row_ct = transaction.execute_update(
            "DELETE FROM key_benefits WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} record(s) deleted.".format(row_ct)) 
        
        
        row_ct = transaction.execute_update(
            "DELETE FROM pricing_plans WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} record(s) deleted.".format(row_ct)) 
        
        
        row_ct = transaction.execute_update(
            "DELETE FROM pricing_plan_features WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} record(s) deleted.".format(row_ct))

database.run_in_transaction(delete_app)

1 record(s) deleted.
1 record(s) deleted.
1 record(s) deleted.
1 record(s) deleted.
1 record(s) deleted.


In [100]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT apps_cat_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

In [101]:
!{CLI_CONNECT} -e "show create table apps_categories"

Table	Create Table
apps_categories	CREATE TABLE apps_categories (
  app_id STRING(50),
  category_id STRING(50),
  CONSTRAINT apps_cat_app_id_fk FOREIGN KEY(app_id) REFERENCES apps(id),
) PRIMARY KEY(app_id, category_id)


In [102]:
!{CLI_CONNECT} -e "CREATE INDEX app_rating_idx ON apps(rating)"